<a href="https://colab.research.google.com/github/jamieewong/cornfutures/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
import os
import numpy as np




In [ ]:
import pandas as pd

In [ ]:
# preprocess = transforms.Compose([
  #  transforms.Resize((224, 224)),
 #   transforms.ToTensor(),

#])

#def load_image(path):
    #img = Image.open(path).convert('RGB')
   # return preprocess(img).unsqueeze(0)  # add batch dim


preprocess = transforms.Compose([
    transforms.ToTensor(),  # Converts to [C,H,W] tensor in [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Standard ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

def load_image(path):
    img = Image.open(path).convert('RGB')  # Ensure 3-channel RGB
    tensor = preprocess(img).unsqueeze(0)  # Add batch dimension [1,3,224,224]
    return tensor

In [ ]:
resnet = models.resnet50(pretrained=True)
resnet.fc = nn.Identity()  # Output vector will be size 2048

resnet = resnet.to('cuda' if torch.cuda.is_available() else 'cpu')
resnet.eval()  # set to eval mode

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 209MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

img_path = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Satellite/area_0/area_0_2018-03-21.png'
input_tensor = load_image(img_path).to(device)

with torch.no_grad():
    features = resnet(input_tensor)  # features shape: [1, 2048]

print(features.shape)
print("Feature vector:", features.cpu().numpy())

torch.Size([1, 2048])
Feature vector: [[0.25888988 0.7675874  0.10958606 ... 0.07674737 0.00695726 0.303851  ]]


In [ ]:
BASE_DIR = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Satellite-RGB'

# assumes you already defined:
# - device  (torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# - resnet  (your ResNet encoder returning 2048-D vectors)
# - load_image(path) → tensor shape (1,3,H,W)  (or (1,1,H,W) if NDVI)

for root, dirs, files in os.walk(BASE_DIR):
    # skip deleted folders
    if 'deleted' in root.lower():
        continue

    rows = []
    png_found = False

    for fname in sorted(files):
        if fname.lower().endswith('.png'):
            png_found = True
            img_path = os.path.join(root, fname)

            # ----- 1️⃣ Extract date from name like "area_2_2018-07-09.png"
            # split on underscores and take the last element, drop extension
            date_str = os.path.splitext(fname)[0].split('_')[-1]

            # ----- 2️⃣ Load image and compute features -----
            input_tensor = load_image(img_path).to(device)
            with torch.no_grad():
                features = resnet(input_tensor)  # expect (1,2048)

            # flatten to (2048,)
            features = features.squeeze().cpu().numpy()
            row = [date_str] + features.tolist()
            rows.append(row)

    # ----- 3️⃣ Write one CSV per folder -----
    if png_found and len(rows) > 0:
        csv_path = os.path.join(root, 'features.csv')
        header = ['Date'] + [f'feat_{i:04d}' for i in range(len(rows[0]) - 1)]

        with open(csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerows(rows)

        print(f"✅ Wrote {len(rows)} feature rows to {csv_path}")


NameError: name 'csv' is not defined

In [ ]:
BASE_DIR = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Satellite-RGB'
area_dirs = [os.path.join(BASE_DIR, d) for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d)) and 'deleted' not in d.lower()]

FUTURES_PATH = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Futures/futures_data.csv'

fut_df = pd.read_csv(FUTURES_PATH)
# make sure the column is called 'Date' and parse it
fut_df['Date'] = pd.to_datetime(fut_df['Date'])
# sort just in case
fut_df = fut_df.sort_values('Date').reset_index(drop=True)
# set index for fast lookup
fut_df = fut_df.set_index('Date')

fut_df.head()

In [ ]:
import os
import pandas as pd

BASE_DIR = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Satellite-RGB'

# collect all CSVs recursively
csv_paths = []
for root, dirs, files in os.walk(BASE_DIR):
    for f in files:
        if f.lower() == "features.csv":
            csv_paths.append(os.path.join(root, f))

print(f"Found {len(csv_paths)} feature CSVs")

# load and concatenate
dfs = []
for path in csv_paths:
    area_name = os.path.basename(os.path.dirname(path))  # e.g. 'area_2'
    df = pd.read_csv(path)

    # ensure 'Date' is first column
    if 'Date' not in df.columns:
        continue
    df = df[['Date'] + [c for c in df.columns if c != 'Date']]

    # insert new column right after Date
    df.insert(1, 'Area', area_name)

    #add column 'Time Delta' to represent time (in days) between photos
    df['Time Delta'] = df['Date'].diff()
    df['Time Delta'] = df['Delta'].astype(int)
    df['Time Delta'] = df['Time Delta']/(8.64*(10**13))
    df['Time Delta'] = df['Time Delta'].astype(int)

    dfs.append(df)

# combine all areas into one DataFrame
all_features_df = pd.concat(dfs, ignore_index=True)

print("✅ Combined dataframe shape:", all_features_df.shape)
display(all_features_df.tail())


Price change evaluation function

In [ ]:
import pandas as pd

def get_future_price_change(fut_df, start_date, offset_days):

    start_date = pd.to_datetime(start_date)

    # 1. FIND FIRST DATE >= start_date
    # Gets the closest available price datafram from the date
    idx = fut_df.index.searchsorted(start_date, side="right") - 1
    if idx < 0:
        return {"No price data on or before starting date."}

    # d1 is the dataframe of a selected date
    d1 = fut_df.index[idx]
    # p1 is pulling the close price for the date
    p1 = fut_df.loc[d1, "Close"]

    # 2. FIND FIRST DATE >= (start_date + offset)
    target_end = start_date + pd.Timedelta(days=offset_days)
    idx2 = fut_df.index.searchsorted(target_end)
    if idx2 >= len(fut_df):
        return {"No price data on or after offset date."}

    d2 = fut_df.index[idx2]
    p2 = fut_df.loc[d2, "Close"]

    # 3. price calculation
    change = p2 - p1
    pct_change = change / p1 * 100 if p1 != 0 else None

    # output
    return {
        "start_date_requested": start_date.date(),
        "start_date_used": d1.date(),
        "start_price": float(p1),

        "end_date_requested": target_end.date(),
        "end_date_used": d2.date(),
        "end_price": float(p2),

        "price_change": float(change),
        "percent_change": float(pct_change),
    }

Testing cells for price calculation function

In [ ]:
# example where start date does not have price, using closest before date
get_future_price_change(fut_df, "2017-07-4", 8)

In [ ]:
# example where start date and offset date have an available price
get_future_price_change(fut_df, "2017-07-3", 8)

In [ ]:
# example where off set date does not have price, using closest price after offset date
get_future_price_change(fut_df, "2017-07-3", 12)

In [ ]:
FUTURES_PATH = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Futures/futures_data.csv'
fut_df = pd.read_csv(FUTURES_PATH)
fut_df['Date'] = pd.to_datetime(fut_df['Date'])
fut_df = fut_df.sort_values('Date').set_index('Date')

def price_on_or_after(date, df=fut_df, col='Close', max_lookahead=5):
    if date in df.index:
        return float(df.loc[date, col])
    for i in range(1, max_lookahead+1):
        d = date + pd.Timedelta(days=i)
        if d in df.index:
            return float(df.loc[d, col])
    return None

In [ ]:
BASE_DIR = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Satellite-RGB'

# discover area folders (skip 'deleted')
area_dirs = [os.path.join(BASE_DIR, d) for d in os.listdir(BASE_DIR)
             if os.path.isdir(os.path.join(BASE_DIR, d)) and 'deleted' not in d.lower()]

region2id = {os.path.basename(p): i for i, p in enumerate(sorted(area_dirs))}
print("Areas:", len(region2id), region2id)

def load_area_features(area_path):
    df = pd.read_csv(os.path.join(area_path, 'features.csv'))
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)
    feat_cols = [c for c in df.columns if c.startswith('feat_')]
    return df[['Date'] + feat_cols], feat_cols

def build_samples(area_dirs, window_size=6, horizon_days=14):
    samples = []
    for area_path in area_dirs:
        area_name = os.path.basename(area_path)
        csv_path = os.path.join(area_path, 'features.csv')
        if not os.path.exists(csv_path):
            continue

        df, feat_cols = load_area_features(area_path)
        if len(df) < window_size:
            continue

        for i in range(len(df) - window_size + 1):
            window = df.iloc[i:i+window_size]
            dates = window['Date'].to_numpy()
            feats = window[feat_cols].to_numpy(dtype=np.float32)  # (T, 2048)

            last_date = dates[-1]
            price_last = price_on_or_after(last_date)                        # Close(t_last)
            price_future = price_on_or_after(last_date + pd.Timedelta(days=horizon_days))  # Close(t_last+H)
            if price_last is None or price_future is None:
                continue
            price_last = price_on_or_after(last_date)

            target_diff = price_future - price_last                          # ΔPrice

            # days since previous image per step
            day_ints = dates.astype('datetime64[D]').astype(int)
            dts = np.diff(day_ints)
            dts = np.insert(dts, 0, 0).astype(np.float32)                    # (T,)

            samples.append({
    'feats': feats,            # (T, 2048)
    'dts': dts,                # (T,)
    'region_id': region2id[area_name],
    'target_diff': float(target_diff),
    'last_date': last_date,
    'area': area_name,
    'price_last': float(price_last),
})
    return samples

# Build your training examples
samples = build_samples(area_dirs, window_size=4, horizon_days=45)
print("Total samples:", len(samples))

In [ ]:
class LSTMDataset(Dataset):
    def __init__(self, samples, normalize_target=True):
        self.samples = samples
        self.normalize_target = normalize_target
        self.mean = np.mean([s['target_diff'] for s in samples]) if normalize_target else 0.0
        self.std  = np.std([s['target_diff'] for s in samples]) + 1e-6 if normalize_target else 1.0

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
      s = self.samples[idx]
      feats = torch.tensor(s['feats'], dtype=torch.float32)      # (T, 2048)
      dts   = torch.tensor(s['dts'], dtype=torch.float32)        # (T,)
      rid   = torch.tensor(s['region_id'], dtype=torch.long)

    # normalize target diff
      y = torch.tensor((s['target_diff'] - self.mean)/self.std, dtype=torch.float32)

      price_last = torch.tensor(s['price_last'], dtype=torch.float32)  # scalar

      last_date_int = np.datetime64(s['last_date']).astype('datetime64[D]').astype(int)

    # return price_last as extra feature
      return feats, dts, rid, y, last_date_int, price_last



In [ ]:
dataset = LSTMDataset(samples, normalize_target=True)

from torch.utils.data import random_split, DataLoader

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=32, shuffle=False)


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.preprocessing import StandardScaler

# ============================================================
# CONFIGURATION
# ============================================================
BASE_DIR = '/content/drive/MyDrive/MLSN-Team-12/Data/Satellite-RGB'
FUTURES_PATH = '/content/drive/MyDrive/MLSN-Team-12/Data/Futures/futures_data.csv'
WEATHER_DIR = '/content/drive/MyDrive/MLSN-Team-12/Data/Weather_Base'
SAVE_DIR = '/content'

SEQ_LEN = 21
PREDICTION_OFFSET_DAYS = 14

# Tuning (Config C - The Winner)
LSTM_HIDDEN = 64
LSTM_LAYERS = 2
DROPOUT = 0.4
LR = 1e-4
WEIGHT_DECAY = 1e-3

BATCH_SIZE = 64
EPOCHS = 60
PATIENCE = 15

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ============================================================
# 1. FEATURE ENGINEERING (ADDED SEASONALITY)
# ============================================================
def add_seasonality(df, date_col='Date'):
    # IMPROVEMENT 2: Cyclic Seasonality
    # Encodes "Day of Year" as coordinates on a circle.
    # Close dates (Dec 31 and Jan 1) become close in value.
    day = df[date_col].dt.dayofyear
    df['Day_Sin'] = np.sin(2 * np.pi * day / 365.0)
    df['Day_Cos'] = np.cos(2 * np.pi * day / 365.0)
    return df

def load_and_engineer_futures(path):
    fut_df = pd.read_csv(path)
    fut_df['Date'] = pd.to_datetime(fut_df['Date'])
    fut_df = fut_df.sort_values('Date').reset_index(drop=True)

    # Add Seasonality
    fut_df = add_seasonality(fut_df)

    if 'Close Percent Change' in fut_df.columns:
        fut_df.rename(columns={'Close Percent Change': 'Close_pct_change'}, inplace=True)
    if 'Close_pct_change' not in fut_df.columns:
        fut_df['Close_pct_change'] = fut_df['Close'].pct_change() * 100.0

    if 'Volume' in fut_df.columns:
        fut_df['Volume_log'] = np.log1p(fut_df['Volume'])
    else:
        fut_df['Volume_log'] = 0.0

    delta = fut_df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    fut_df['RSI'] = 100 - (100 / (1 + rs))
    fut_df['RSI'] = fut_df['RSI'].fillna(50)

    ema12 = fut_df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = fut_df['Close'].ewm(span=26, adjust=False).mean()
    fut_df['MACD'] = ema12 - ema26

    sma20 = fut_df['Close'].rolling(window=20).mean()
    std20 = fut_df['Close'].rolling(window=20).std()
    bb_upper = sma20 + (std20 * 2)
    bb_lower = sma20 - (std20 * 2)
    fut_df['BB_PctB'] = (fut_df['Close'] - bb_lower) / (bb_upper - bb_lower)
    fut_df['BB_PctB'] = fut_df['BB_PctB'].fillna(0.5)

    return fut_df.dropna().reset_index(drop=True)

def load_and_engineer_weather(weather_dir):
    print(f"Scanning {weather_dir} for weather data...")
    weather_frames = []
    for f in os.listdir(weather_dir):
        if f.endswith('.csv'):
            import re
            match = re.search(r'(\d+)', f)
            if match:
                area_name = f"area_{match.group(1)}"
                path = os.path.join(weather_dir, f)
                try:
                    df = pd.read_csv(path)
                    date_col = next((c for c in df.columns if 'date' in c.lower()), None)
                    if not date_col: continue
                    df = df.rename(columns={date_col: 'Date'})
                    df['Date'] = pd.to_datetime(df['Date'])
                    df = df.sort_values('Date')

                    t_mean = (df['temp_max_C'] + df['temp_min_C']) / 2
                    df['GDD'] = (t_mean - 10).clip(lower=0)
                    df['GDD_Cum_7d'] = df['GDD'].rolling(7).sum()
                    df['Precip_Sum_7d'] = df['precip_mm'].rolling(7).sum()

                    cols_to_keep = ['Date', 'GDD_Cum_7d', 'Precip_Sum_7d']
                    df = df[cols_to_keep].copy()
                    df['Area'] = area_name
                    weather_frames.append(df)
                except Exception: continue

    if not weather_frames: raise ValueError("No weather data found!")
    all_weather = pd.concat(weather_frames, ignore_index=True)

    cols_to_fill = [c for c in all_weather.columns if c not in ['Area', 'Date']]
    all_weather[cols_to_fill] = all_weather.groupby('Area')[cols_to_fill].transform(lambda x: x.ffill().bfill())
    return all_weather

def load_satellite_data(base_dir):
    print(f"Scanning {base_dir} for features.csv...")
    csv_paths = []
    for root, dirs, files in os.walk(base_dir):
        for f in files:
            if f.lower() == "features.csv":
                csv_paths.append(os.path.join(root, f))

    dfs = []
    for path in csv_paths:
        area_name = os.path.basename(os.path.dirname(path))
        try:
            df = pd.read_csv(path)
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                df = df.dropna(subset=['Date'])
                df = df[['Date'] + [c for c in df.columns if c != 'Date']]
                df.insert(1, 'Area', area_name)
                df = df.sort_values('Date')
                df['Time Delta'] = df['Date'].diff().dt.total_seconds().div(86400).fillna(0).astype(int)
                dfs.append(df)
        except Exception: continue

    all_features_df = pd.concat(dfs, ignore_index=True)
    all_features_df = all_features_df.sort_values('Date').reset_index(drop=True)
    feat_cols = [c for c in all_features_df.columns if c.startswith('feat_')]
    agg_dict = {c: 'mean' for c in feat_cols}
    all_features_df = all_features_df.groupby(['Area', 'Date', 'Time Delta'], as_index=False).agg(agg_dict)
    return all_features_df

# ============================================================
# 2. DATASET & MODEL
# ============================================================
class CornSequenceDataset(Dataset):
    def __init__(self, features_df, fut_price_df, img_cols, fut_cols,
                 seq_len=SEQ_LEN, prediction_offset=PREDICTION_OFFSET_DAYS):

        self.features_df = features_df
        self.fut_price_df = fut_price_df
        self.img_cols = img_cols
        self.fut_cols = fut_cols
        self.seq_len = seq_len
        self.prediction_offset = prediction_offset
        self.area_to_id = {a: i for i, a in enumerate(sorted(features_df['Area'].unique()))}
        self.area_groups = {area: group for area, group in features_df.groupby('Area')}

        self.label_mean = 0.0
        self.label_std = 1.0
        self.label_transform = None
        self.feature_transform = None

        self.sequences = []
        self._build_sequences()

    def _build_sequences(self):
        print(f"Building sequences (L={self.seq_len})...")
        for area in self.area_groups:
            area_data = self.area_groups[area].sort_values('Date').reset_index(drop=True)
            n = len(area_data)
            for i in range(0, n - self.seq_len + 1):
                seq_end_idx = i + self.seq_len
                last_img_date = area_data.iloc[seq_end_idx - 1]['Date']
                target_ideal_date = last_img_date + pd.Timedelta(days=self.prediction_offset)
                try:
                    loc = self.fut_price_df.index.get_indexer([target_ideal_date], method='nearest')[0]
                    if loc == -1: continue
                    actual_target_date = self.fut_price_df.index[loc]
                    if abs((actual_target_date - target_ideal_date).days) > 5: continue
                    if last_img_date in self.fut_price_df.index:
                        self.sequences.append({
                            'area': area,
                            'data_indices': (i, seq_end_idx),
                            'base_date': last_img_date,
                            'target_date': actual_target_date
                        })
                except: continue
        self.sequences.sort(key=lambda x: x['target_date'])

    def _compute_label(self, idx):
        seq = self.sequences[idx]
        p_base = float(self.fut_price_df.loc[seq['base_date'], 'Close'])
        p_target = float(self.fut_price_df.loc[seq['target_date'], 'Close'])
        raw_pct = (p_target - p_base) / p_base * 100.0
        if self.label_transform: return self.label_transform(raw_pct)
        return raw_pct

    def __len__(self): return len(self.sequences)

    def __getitem__(self, idx):
        seq_info = self.sequences[idx]
        start, end = seq_info['data_indices']
        subset = self.area_groups[seq_info['area']].iloc[start:end]
        img = torch.FloatTensor(subset[self.img_cols].values)
        fut_raw = subset[self.fut_cols].values
        if self.feature_transform: fut_raw = self.feature_transform(fut_raw)
        fut = torch.FloatTensor(fut_raw)
        aid = torch.LongTensor([self.area_to_id[seq_info['area']]] * self.seq_len)
        time = torch.FloatTensor(subset['Time Delta'].values.reshape(-1, 1))
        lbl = torch.FloatTensor([self._compute_label(idx)])
        return img, fut, aid, time, lbl

class CornETFModelFinal(nn.Module):
    def __init__(self, img_dim, fut_dim, num_areas):
        super().__init__()
        self.img_bottleneck = nn.Sequential(
            nn.Linear(img_dim, 64),
            nn.ReLU(),
            nn.LayerNorm(64),
            nn.Dropout(0.1)
        )
        self.fut_norm = nn.LayerNorm(fut_dim)
        self.area_embed = nn.Embedding(num_areas, 16)
        self.time_embed = nn.Linear(1, 8)

        self.lstm_input_dim = 64 + fut_dim + 16 + 8

        self.lstm = nn.LSTM(
            input_size=self.lstm_input_dim,
            hidden_size=LSTM_HIDDEN,
            num_layers=LSTM_LAYERS,
            batch_first=True,
            dropout=DROPOUT
        )
        self.dropout = nn.Dropout(DROPOUT)
        self.head = nn.Linear(LSTM_HIDDEN, 1)

    def forward(self, img, fut, aid, time):
        img_c = self.img_bottleneck(img)
        fut_c = self.fut_norm(fut)
        area_v = self.area_embed(aid)
        time_v = self.time_embed(time)
        x = torch.cat([img_c, fut_c, area_v, time_v], dim=2)
        out, _ = self.lstm(x)
        return self.head(self.dropout(out[:, -1, :]))

# ============================================================
# 3. EXECUTION
# ============================================================
def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    if not batch: return None
    return (torch.stack([b[0] for b in batch]), torch.stack([b[1] for b in batch]),
            torch.stack([b[2] for b in batch]), torch.stack([b[3] for b in batch]),
            torch.stack([b[4] for b in batch]))

def compute_normalization(dataset, train_idx):
    labels = [dataset._compute_label(i) for i in train_idx]
    labels = np.array(labels)
    mean, std = float(labels.mean()), float(labels.std())
    dataset.label_mean = mean
    dataset.label_std = max(std, 2.0)
    dataset.label_transform = lambda x: (x - mean) / (dataset.label_std + 1e-8)
    print(f"✅ Label Norm: Mean={mean:.4f}, Std={dataset.label_std:.4f}")

if __name__ == '__main__':
    print(f"Using Device: {DEVICE}")
    fut_df = load_and_engineer_futures(FUTURES_PATH)
    fut_price_df = fut_df.set_index('Date')[['Close']].sort_index()
    sat_df = load_satellite_data(BASE_DIR)
    weather_df = load_and_engineer_weather(WEATHER_DIR)

    sat_weather_df = pd.merge(sat_df, weather_df, on=['Area', 'Date'], how='left')
    cols_to_fill = [c for c in sat_weather_df.columns if c not in ['Area', 'Date']]
    sat_weather_df[cols_to_fill] = sat_weather_df.groupby('Area')[cols_to_fill].transform(lambda x: x.ffill().bfill())
    merged_df = pd.merge(sat_weather_df, fut_df, on='Date', how='inner', suffixes=('', '_fut'))

    img_cols = [c for c in merged_df.columns if c.startswith('feat_')]
    fut_cols = [c for c in merged_df.columns if c not in ['Date', 'Area', 'Time Delta'] and not c.startswith('feat_')]

    dataset = CornSequenceDataset(merged_df, fut_price_df, img_cols, fut_cols, seq_len=SEQ_LEN)
    all_dates = pd.to_datetime([seq['target_date'] for seq in dataset.sequences])

    folds = [
        {'name': 'Fold 1: The Rally (Test 2021)', 'train_end': '2021-01-01', 'test_end': '2022-01-01'},
        {'name': 'Fold 2: The Peak (Test 2022)',  'train_end': '2022-01-01', 'test_end': '2023-01-01'},
        {'name': 'Fold 3: The Crash (Test 2023)', 'train_end': '2023-01-01', 'test_end': '2024-01-01'}
    ]

    final_results = []

    for fold in folds:
        print(f"\n>>> RUNNING: {fold['name']}")

        train_mask = all_dates < pd.Timestamp(fold['train_end'])
        test_mask = (all_dates >= pd.Timestamp(fold['train_end'])) & (all_dates < pd.Timestamp(fold['test_end']))
        train_idx = np.where(train_mask)[0]
        val_idx = np.where(test_mask)[0]

        if len(val_idx) < 50: continue

        compute_normalization(dataset, train_idx)

        scaler = StandardScaler()
        train_features = []
        for i in train_idx[::10]:
            seq_info = dataset.sequences[i]
            start, end = seq_info['data_indices']
            subset = dataset.area_groups[seq_info['area']].iloc[start:end]
            train_features.append(subset[dataset.fut_cols].values)
        scaler.fit(np.vstack(train_features))
        dataset.feature_transform = lambda x: scaler.transform(x)

        train_loader = DataLoader(Subset(dataset, train_idx), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
        val_loader = DataLoader(Subset(dataset, val_idx), batch_size=64, shuffle=False, collate_fn=collate_fn)

        model = CornETFModelFinal(
            img_dim=len(img_cols), fut_dim=len(fut_cols), num_areas=len(dataset.area_to_id)
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

        # IMPROVEMENT 1: Huber Loss (Robust Regression)
        # Acts like MSE for small errors, but linear for outliers.
        # Prevents explosion when the model misses a crash.
        criterion = nn.HuberLoss(delta=1.0)

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

        best_val = float('inf')
        patience_counter = 0

        for epoch in range(EPOCHS):
            model.train()
            t_loss = 0
            for img, fut, aid, time, lbl in train_loader:
                img, fut, aid, time, lbl = img.to(DEVICE), fut.to(DEVICE), aid.to(DEVICE), time.to(DEVICE), lbl.to(DEVICE)
                preds = model(img, fut, aid, time)
                loss = criterion(preds, lbl)
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                t_loss += loss.item()

             model.eval()
            v_loss = 0
            with torch.no_grad():
                for img, fut, aid, time, lbl in val_loader:
                    img, fut, aid, time, lbl = img.to(DEVICE), fut.to(DEVICE), aid.to(DEVICE), time.to(DEVICE), lbl.to(DEVICE)
                    v_loss += criterion(model(img, fut, aid, time), lbl).item()

            avg_v = v_loss/len(val_loader)
            scheduler.step(avg_v)

            if avg_v < best_val:
                best_val = avg_v
                patience_counter = 0
                torch.save(model.state_dict(), f"{SAVE_DIR}/best_model_{fold['name'].split(':')[0].replace(' ', '_')}.pt")
            else:
                patience_counter += 1

            if (epoch+1) % 5 == 0:
                print(f"Fold Epoch {epoch+1}: Train={t_loss/len(train_loader):.4f} | Val={avg_v:.4f}")

            if patience_counter >= PATIENCE: break

        results_table.append({'Fold': fold['name'], 'Best_MSE': best_val})

    print("\n📊 FINAL REPORT (Lowest Loss Wins)")
    for r in results_table:
        print(f"{r['Fold']:<30} | Huber Loss: {r['Best_MSE']:.4f}")

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# --- Configuration (Must match your training config) ---
SEQ_LEN = 21
LSTM_HIDDEN = 64
LSTM_LAYERS = 2
DROPOUT = 0.4
FUT_DIM = 10   # Approx columns: Open, High, Low, Close, Vol, OI, RSI, MACD, BB, BB_width etc.
              # Adjust this if your feature count differs (e.g. 9 or 10)
NUM_AREAS = 6 # Based on region2id count

class CornPriceModel(nn.Module):
    def __init__(self, num_areas=NUM_AREAS, fut_dim=FUT_DIM):
        super().__init__()

        # 1. Image Feature Bottleneck (ResNet 2048 -> 64)
        self.img_bottleneck = nn.Sequential(
            nn.Linear(2048, 64),
            nn.ReLU()
        )

        # 2. Future Feature Normalization
        # Using BatchNorm1d. Expects (N, C, L), so we permute in forward.
        self.fut_norm = nn.BatchNorm1d(fut_dim)

        # 3. Embeddings
        self.area_embed = nn.Embedding(num_areas, 16)
        self.time_embed = nn.Embedding(12, 8) # Month 0-11

        # 4. LSTM
        # Input = img(64) + fut(fut_dim) + area(16) + time(8)
        self.lstm_input_dim = 64 + fut_dim + 16 + 8

        self.lstm = nn.LSTM(
            input_size=self.lstm_input_dim,
            hidden_size=LSTM_HIDDEN,
            num_layers=LSTM_LAYERS,
            batch_first=True,
            dropout=DROPOUT
        )

        self.dropout = nn.Dropout(DROPOUT)
        self.head = nn.Linear(LSTM_HIDDEN, 1)

    def forward(self, img, fut, aid, time):
        # img: (Batch, Seq, 2048)
        # fut: (Batch, Seq, Fut_Dim)
        # aid: (Batch,)
        # time: (Batch,)

        # Process Images
        img_c = self.img_bottleneck(img) # -> (Batch, Seq, 64)

        # Process Futures (Permute for BN1d: N, Seq, C -> N, C, Seq)
        fut_perm = fut.permute(0, 2, 1)
        fut_c = self.fut_norm(fut_perm)
        fut_c = fut_c.permute(0, 2, 1)   # -> (Batch, Seq, C)

        # Process Embeddings (Expand to Sequence Length)
        area_v = self.area_embed(aid).unsqueeze(1).expand(-1, SEQ_LEN, -1)
        time_v = self.time_embed(time).unsqueeze(1).expand(-1, SEQ_LEN, -1)

        # Concatenate
        x = torch.cat([img_c, fut_c, area_v, time_v], dim=2)

        # LSTM & Head
        out, _ = self.lstm(x)
        return self.head(self.dropout(out[:, -1, :]))

print("✅ Model Class Defined")

In [ ]:
# --- User Settings ---
MODEL_PATH = 'path.pt'  # Replace with your downloaded .pt file name
LABEL_MEAN = 0.0532                  # REPLACE with the Mean from your training log
LABEL_STD = 2.1245                   # REPLACE with the Std from your training log

# --- Setup ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CornPriceModel(num_areas=NUM_AREAS, fut_dim=FUT_DIM).to(device)

# --- Load Weights ---
try:
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    print(f"✅ Loaded weights from {MODEL_PATH}")
except FileNotFoundError:
    print(f"❌ File {MODEL_PATH} not found. Initializing with random weights for demo.")
model.eval()

# --- Create Dummy Inputs (Batch Size = 1) ---
# Simulating: 1 sample, 21 days history
dummy_img = torch.randn(1, SEQ_LEN, 2048).to(device)     # Random ResNet features
dummy_fut = torch.randn(1, SEQ_LEN, FUT_DIM).to(device)  # Random Futures data
dummy_aid = torch.tensor([0]).to(device)                 # Area ID 0
dummy_time = torch.tensor([5]).to(device)                # Month 5 (June)

# --- Simulation Context ---
last_known_price = 450.00  # Price at day t
real_future_price = 462.50 # The "Real" price at day t+14

# --- Run Prediction ---
with torch.no_grad():
    # Model outputs normalized price difference
    pred_norm = model(dummy_img, dummy_fut, dummy_aid, dummy_time).item()

# --- De-normalize ---
# Formula: (Output * Std) + Mean = Predicted_Diff
pred_diff = (pred_norm * LABEL_STD) + LABEL_MEAN
predicted_price = last_known_price + pred_diff

# --- Output Report ---
print(f"\n{'='*40}")
print(f"🌽 CORN PRICE PREDICTION DEMO")
print(f"{'='*40}")
print(f"Input State")
print(f"  - Sequence Length : {SEQ_LEN} days")
print(f"  - Last Known Price: ${last_known_price:.2f}")
print(f"----------------------------------------")
print(f"Model Inference")
print(f"  - Raw Output      : {pred_norm:.4f}")
print(f"  - Predicted Diff  : ${pred_diff:.2f}")
print(f"----------------------------------------")
print(f"Results")
print(f"  - PREDICTED PRICE : ${predicted_price:.2f}")
print(f"  - REAL PRICE      : ${real_future_price:.2f}")
print(f"  - Absolute Error  : ${abs(predicted_price - real_future_price):.2f}")
print(f"{'='*40}")

In [ ]:
import os
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# --- Configuration ---
AREA_NAME = 'area_51'   # The folder name
YEAR = '2021'           # The year to filter by
SEQ_LEN = 21            # Sequence length model expects

# Path to your data (Adjust if needed!)
IMG_BASE_DIR = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Satellite-RGB'
FUTURES_PATH = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12/Data/Futures/futures.csv'
# ^ Check this path! It might be named differently in your drive.

# --- Image Preprocessing (Standard ImageNet) ---
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_real_data_sequence(area_name, year, seq_len):
    """Loads images and futures for a specific area and year."""

    # 1. Find Images
    area_dir = os.path.join(IMG_BASE_DIR, area_name)
    if not os.path.exists(area_dir):
        raise FileNotFoundError(f"Directory not found: {area_dir}")

    all_files = sorted([f for f in os.listdir(area_dir) if f.endswith('.png')])

    # Filter for the specific year
    target_files = [f for f in all_files if year in f]

    if len(target_files) < seq_len:
        print(f"⚠️ Warning: Only found {len(target_files)} images for {year}. Need {seq_len}.")
        # Fallback: take last available images regardless of year if needed, or just return what we have
        target_files = all_files[-seq_len:]
    else:
        # Take the LAST sequence of the year (most recent data for that year)
        target_files = target_files[-seq_len:]

    print(f"✅ Found {len(target_files)} images. Date range: {target_files[0]} to {target_files[-1]}")

    # 2. Process Images -> ResNet Features
    # Ensure 'resnet' is defined and on device (from your Main.ipynb)
    resnet.eval()
    img_feats = []
    dates = []

    print("📸 Processing images through ResNet...")
    with torch.no_grad():
        for fname in target_files:
            # Parse Date
            date_str = os.path.splitext(fname)[0].split('_')[-1]
            dates.append(date_str)

            # Load & Process
            path = os.path.join(area_dir, fname)
            img = Image.open(path).convert('RGB')
            input_tensor = preprocess(img).unsqueeze(0).to(device)

            # Extract Feature
            feat = resnet(input_tensor) # (1, 2048)
            img_feats.append(feat)

    # Stack into (1, Seq_Len, 2048)
    img_tensor = torch.stack(img_feats, dim=1)

    # 3. Load & Align Futures
    print("📉 Loading Futures data...")
    fut_df = pd.read_csv(FUTURES_PATH)

    # Ensure dates are strings for matching
    fut_df['Date'] = fut_df['Date'].astype(str)

    # Select rows matching our image dates
    # Note: This assumes columns 1 onwards are the features (0 is Date)
    # You might need to adjust column slicing [:, 1:] if your CSV structure is different
    matched_data = []
    for d in dates:
        row = fut_df[fut_df['Date'] == d]
        if not row.empty:
            # Drop Date column and convert to float
            data_row = row.iloc[0, 1:].values.astype(float)
            matched_data.append(data_row)
        else:
            print(f"⚠️ Missing futures data for {d}, filling with zeros.")
            matched_data.append(np.zeros(10)) # Assuming 10 dims

    fut_tensor = torch.tensor(matched_data, dtype=torch.float32).unsqueeze(0).to(device)

    return img_tensor, fut_tensor, dates

print("✅ Data Loader Ready")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Data Loader Ready


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import pickle

# ============================================================
# 1. CONFIGURATION
# ============================================================
BASE_PATH = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12'
MODEL_PATH = os.path.join(BASE_PATH, 'Models/best_model_Fold_1.pt')
SCALAR_PATH = os.path.join(BASE_PATH, 'Models/scalar_Fold_1.pkl')
WEATHER_DIR = os.path.join(BASE_PATH, 'Data/Weather_Base')
FUTURES_PATH = os.path.join(BASE_PATH, 'Data/Futures/futures_data.csv')
AREA_FEATURE_PATH = os.path.join(BASE_PATH, 'Data/Satellite-RGB/area_16/features.csv')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SEQ_LEN = 21
NUM_AREAS = 190

# ============================================================
# 2. MODEL ARCHITECTURE
# ============================================================
class CornETFModelFinal(nn.Module):
    def __init__(self, img_dim, fut_dim, num_areas):
        super().__init__()
        self.img_bottleneck = nn.Sequential(
            nn.Linear(img_dim, 64),
            nn.ReLU(),
            nn.LayerNorm(64),
            nn.Dropout(0.1)
        )
        self.fut_norm = nn.LayerNorm(fut_dim)
        self.area_embed = nn.Embedding(num_areas, 16)
        self.time_embed = nn.Linear(1, 8)

        self.lstm_input_dim = 64 + fut_dim + 16 + 8
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=64, num_layers=2, batch_first=True, dropout=0.4)
        self.dropout = nn.Dropout(0.4)
        self.head = nn.Linear(64, 1)

    def forward(self, img, fut, aid, time):
        img_c = self.img_bottleneck(img)
        fut_c = self.fut_norm(fut)
        area_v = self.area_embed(aid)
        time_v = self.time_embed(time)
        x = torch.cat([img_c, fut_c, area_v, time_v], dim=2)
        out, _ = self.lstm(x)
        return self.head(self.dropout(out[:, -1, :]))

# ============================================================
# 3. FEATURE ENGINEERING
# ============================================================
def engineer_futures(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    # Seasonality
    day = df['Date'].dt.dayofyear
    df['Day_Sin'] = np.sin(2 * np.pi * day / 365.0)
    df['Day_Cos'] = np.cos(2 * np.pi * day / 365.0)

    # Missing Columns check
    if 'Close Percent Change' in df.columns:
        df = df.rename(columns={'Close Percent Change': 'Close_pct_change'})
    if 'Close_pct_change' not in df.columns:
        df['Close_pct_change'] = df['Close'].pct_change() * 100.0
    if 'Volume' in df.columns:
        df['Volume_log'] = np.log1p(df['Volume'])
    else:
        df['Volume_log'] = 0.0

    # Technicals
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['RSI'] = df['RSI'].fillna(50)

    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26

    sma20 = df['Close'].rolling(window=20).mean()
    std20 = df['Close'].rolling(window=20).std()
    bb_upper = sma20 + (std20 * 2)
    bb_lower = sma20 - (std20 * 2)
    df['BB_PctB'] = (df['Close'] - bb_lower) / (bb_upper - bb_lower)
    df['BB_PctB'] = df['BB_PctB'].fillna(0.5)

    return df.dropna().reset_index(drop=True)

def get_area_weather(weather_dir, area_name):
    target_file = None
    for f in os.listdir(weather_dir):
        if area_name in f and f.endswith('.csv'):
            target_file = os.path.join(weather_dir, f)
            break
    if not target_file: return None
    df = pd.read_csv(target_file)
    date_col = next((c for c in df.columns if 'date' in c.lower()), None)
    if not date_col: return None
    df = df.rename(columns={date_col: 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    try:
        t_mean = (df['temp_max_C'] + df['temp_min_C']) / 2
        df['GDD'] = (t_mean - 10).clip(lower=0)
        df['GDD_Cum_7d'] = df['GDD'].rolling(7).sum()
        df['Precip_Sum_7d'] = df['precip_mm'].rolling(7).sum()
        return df[['Date', 'GDD_Cum_7d', 'Precip_Sum_7d']].copy()
    except: return None

# ============================================================
# 4. EXECUTION PIPELINE
# ============================================================
print("🔄 Loading & Engineering Data...")

if os.path.exists(SCALAR_PATH):
    with open(SCALAR_PATH, 'rb') as f:
        scalars = pickle.load(f)
    if isinstance(scalars, dict):
        LABEL_MEAN, LABEL_STD = scalars.get('mean', 0), scalars.get('std', 1)
    else:
        LABEL_MEAN, LABEL_STD = scalars[0], scalars[1]
else:
    LABEL_MEAN, LABEL_STD = 0.0, 1.0

# 1. Load Satellite (The "Anchor" Dataset)
sat_df = pd.read_csv(AREA_FEATURE_PATH)
sat_df['Date'] = pd.to_datetime(sat_df['Date'])

# 2. Load Weather
weather_df = get_area_weather(WEATHER_DIR, 'area_16')

# 3. Load & Engineer Futures
fut_raw = pd.read_csv(FUTURES_PATH)
fut_df = engineer_futures(fut_raw)

# 4. MERGE (LEFT JOIN FIX)
# We use left join on sat_df to keep ALL 29 images, even if they fall on weekends.
if weather_df is not None:
    merged = pd.merge(sat_df, weather_df, on='Date', how='left')
    merged[['GDD_Cum_7d', 'Precip_Sum_7d']] = merged[['GDD_Cum_7d', 'Precip_Sum_7d']].ffill().bfill().fillna(0)
else:
    merged = sat_df

# Merge with Futures
final_df = pd.merge(merged, fut_df, on='Date', how='left')

# CRITICAL FIX: Forward fill weekend gaps for market data
final_df = final_df.sort_values('Date').ffill().bfill()

# Filter for 2021
final_df = final_df[final_df['Date'].dt.year == 2018]

if len(final_df) < SEQ_LEN: raise ValueError(f"Not enough data. Found {len(final_df)} rows.")

# Select the LAST 21 images (should now be a partial year, not the whole thing)
seq_df = final_df.iloc[:SEQ_LEN].copy()

# Prepare Tensors
img_cols = [c for c in final_df.columns if c.startswith('feat_')]
exclude = ['Date', 'Area', 'Time Delta'] + img_cols
fut_cols = [c for c in final_df.columns if c not in exclude]

t_img = torch.tensor(seq_df[img_cols].values, dtype=torch.float32).unsqueeze(0).to(DEVICE)
t_fut = torch.tensor(seq_df[fut_cols].values, dtype=torch.float32).unsqueeze(0).to(DEVICE)
t_aid = torch.LongTensor([0] * SEQ_LEN).unsqueeze(0).to(DEVICE)

dates = seq_df['Date'].values
dts = np.diff(dates.astype('datetime64[D]').astype(int))
dts = np.insert(dts, 0, 0).astype(np.float32)
t_time = torch.tensor(dts).reshape(1, SEQ_LEN, 1).to(DEVICE)

# ============================================================
# 5. RUN & REPORT
# ============================================================
print("🏗️  Running Model...")
model = CornETFModelFinal(len(img_cols), len(fut_cols), NUM_AREAS).to(DEVICE)

if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
else:
    print("❌ Model weights not found!")

model.eval()
with torch.no_grad():
    pred_norm = model(t_img, t_fut, t_aid, t_time).item()

pred_diff = (pred_norm * LABEL_STD) + LABEL_MEAN

# --- ACCURACY CHECK ---
last_date = seq_df['Date'].iloc[-1]
target_date = last_date + pd.Timedelta(days=14)

# Find 'Real' price ~14 days later in the full futures data
# We look in fut_df (daily market data) not final_df (sparse satellite data)
future_rows = fut_df[fut_df['Date'] >= target_date]

close_col = next((c for c in fut_cols if 'close' in c.lower() and 'pct' not in c.lower()), None)
last_price = seq_df[close_col].iloc[-1] if close_col else 0.0

if not future_rows.empty:
    real_future_price = future_rows.iloc[0]['Close']
    real_actual_date = future_rows.iloc[0]['Date'].strftime('%Y-%m-%d')
    real_change = real_future_price - last_price
    accuracy_msg = f"${real_change:+.2f}"
    error_val = abs(real_change - pred_diff)
else:
    real_change = 0.0
    accuracy_msg = "N/A"
    error_val = 0.0


print(f"PREDICTED PRICE   : ${last_price + pred_diff:.2f}")
print(f"REAL PRICE        : ${last_price + real_change:.2f}")
print(f"{'='*45}")

🔄 Loading & Engineering Data...
🏗️  Running Model...
PREDICTED PRICE   : $370.87
REAL PRICE        : $374.00


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import pickle

# ============================================================
# 1. CONFIGURATION
# ============================================================
BASE_PATH = '/content/drive/Shareddrives/MLSN-Team-12/MLSN-Team-12'
MODEL_PATH = os.path.join(BASE_PATH, 'Models/best_model_Fold_1.pt')
SCALER_PATH = os.path.join(BASE_PATH, 'Models/scaler_Fold_1.pkl')

WEATHER_DIR = os.path.join(BASE_PATH, 'Data/Weather_Base')
FUTURES_PATH = os.path.join(BASE_PATH, 'Data/Futures/futures_data.csv')
AREA_FEATURE_PATH = os.path.join(BASE_PATH, 'Data/Satellite-RGB/area_16/features.csv')
AREA_ID_INT = 15  # Area 16

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SEQ_LEN = 21
NUM_AREAS = 190
EXPECTED_FUT_DIM = 15 # The model was trained on 15 features. We MUST provide 15.

# ============================================================
# 2. MODEL ARCHITECTURE
# ============================================================
class CornETFModelFinal(nn.Module):
    def __init__(self, img_dim, fut_dim, num_areas):
        super().__init__()
        self.img_bottleneck = nn.Sequential(
            nn.Linear(img_dim, 64),
            nn.ReLU(),
            nn.LayerNorm(64),
            nn.Dropout(0.1)
        )
        self.fut_norm = nn.LayerNorm(fut_dim)
        self.area_embed = nn.Embedding(num_areas, 16)
        self.time_embed = nn.Linear(1, 8)

        self.lstm_input_dim = 64 + fut_dim + 16 + 8
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=64, num_layers=2, batch_first=True, dropout=0.4)
        self.dropout = nn.Dropout(0.4)
        self.head = nn.Linear(64, 1)

    def forward(self, img, fut, aid, time):
        img_c = self.img_bottleneck(img)
        fut_c = self.fut_norm(fut)
        area_v = self.area_embed(aid)
        time_v = self.time_embed(time)
        x = torch.cat([img_c, fut_c, area_v, time_v], dim=2)
        out, _ = self.lstm(x)
        return self.head(self.dropout(out[:, -1, :]))

# ============================================================
# 3. FEATURE ENGINEERING
# ============================================================
def engineer_futures(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    day = df['Date'].dt.dayofyear
    df['Day_Sin'] = np.sin(2 * np.pi * day / 365.0)
    df['Day_Cos'] = np.cos(2 * np.pi * day / 365.0)

    if 'Close Percent Change' in df.columns:
        df = df.rename(columns={'Close Percent Change': 'Close_pct_change'})
    if 'Close_pct_change' not in df.columns:
        df['Close_pct_change'] = df['Close'].pct_change() * 100.0
    if 'Volume' in df.columns:
        df['Volume_log'] = np.log1p(df['Volume'])
    else:
        df['Volume_log'] = 0.0

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['RSI'] = df['RSI'].fillna(50)

    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26

    sma20 = df['Close'].rolling(window=20).mean()
    std20 = df['Close'].rolling(window=20).std()
    bb_upper = sma20 + (std20 * 2)
    bb_lower = sma20 - (std20 * 2)
    df['BB_PctB'] = (df['Close'] - bb_lower) / (bb_upper - bb_lower)
    df['BB_PctB'] = df['BB_PctB'].fillna(0.5)

    return df.dropna().reset_index(drop=True)

def get_area_weather(weather_dir, area_name):
    target_file = None
    for f in os.listdir(weather_dir):
        if area_name in f and f.endswith('.csv'):
            target_file = os.path.join(weather_dir, f)
            break
    if not target_file: return None
    df = pd.read_csv(target_file)
    date_col = next((c for c in df.columns if 'date' in c.lower()), None)
    if not date_col: return None
    df = df.rename(columns={date_col: 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    try:
        t_mean = (df['temp_max_C'] + df['temp_min_C']) / 2
        df['GDD'] = (t_mean - 10).clip(lower=0)
        df['GDD_Cum_7d'] = df['GDD'].rolling(7).sum()
        df['Precip_Sum_7d'] = df['precip_mm'].rolling(7).sum()
        return df[['Date', 'GDD_Cum_7d', 'Precip_Sum_7d']].copy()
    except: return None

# ============================================================
# 4. PREDICTION FUNCTION
# ============================================================
def predict_scenario(target_date_str, description, model, feature_scaler, full_df):
    target_date = pd.Timestamp(target_date_str)
    cutoff_date = target_date - pd.Timedelta(days=14)

    # Filter History
    history = full_df[full_df['Date'] <= cutoff_date].copy()

    if len(history) < SEQ_LEN:
        print(f"⚠️ SKIPPING {target_date_str}: Not enough history found.")
        return

    # Grab Sequence
    seq_df = history.tail(SEQ_LEN).copy()

    # CHECK DATA FRESHNESS
    last_img_date = seq_df['Date'].iloc[-1]
    days_stale = (cutoff_date - last_img_date).days
    freshness_status = "✅ Fresh" if days_stale < 10 else f"⚠️ Stale ({days_stale} days old)"

    # Prepare Tensors
    img_cols = [c for c in full_df.columns if c.startswith('feat_')]
    exclude = ['Date', 'Area', 'Time Delta'] + img_cols
    fut_cols = [c for c in full_df.columns if c not in exclude]

    # --- SAFETY PADDING (THE FIX) ---
    # Model expects 15 features. If we have 13, we pad with zeros.
    raw_fut = seq_df[fut_cols].values
    current_dim = raw_fut.shape[1]

    if current_dim < EXPECTED_FUT_DIM:
        diff = EXPECTED_FUT_DIM - current_dim
        # Create zeros of shape (21, diff)
        zeros = np.zeros((raw_fut.shape[0], diff))
        # Stack them
        raw_fut = np.hstack([raw_fut, zeros])
    # --------------------------------

    # Scale Features
    if feature_scaler:
        scaled_fut = feature_scaler.transform(raw_fut)
    else:
        scaled_fut = raw_fut

    t_img = torch.tensor(seq_df[img_cols].values, dtype=torch.float32).unsqueeze(0).to(DEVICE)
    t_fut = torch.tensor(scaled_fut, dtype=torch.float32).unsqueeze(0).to(DEVICE)
    t_aid = torch.LongTensor([AREA_ID_INT] * SEQ_LEN).unsqueeze(0).to(DEVICE)

    dates = seq_df['Date'].values
    dts = np.diff(dates.astype('datetime64[D]').astype(int))
    dts = np.insert(dts, 0, 0).astype(np.float32)
    t_time = torch.tensor(dts).reshape(1, SEQ_LEN, 1).to(DEVICE)

    # Predict
    with torch.no_grad():
        pred_raw = model(t_img, t_fut, t_aid, t_time).item()

    # Fold 1 Stats
    LABEL_MEAN, LABEL_STD = 0.8693, 4.9082
    pred_pct = (pred_raw * LABEL_STD) + LABEL_MEAN

    # Check Actuals
    last_price = seq_df['Close'].iloc[-1]
    pred_price = last_price * (1 + pred_pct / 100.0)

    # Find real price
    future_data = fut_df[fut_df['Date'] >= target_date]

    print(f"\n📝 SCENARIO: {description}")
    print(f"   Target Date: {target_date.date()} | Last Image Seen: {last_img_date.date()} ({freshness_status})")
    print(f"   AI Prediction: {pred_pct:+.2f}%  (${pred_price:.2f})")

    if not future_data.empty:
        real_price = future_data.iloc[0]['Close']
        real_pct = ((real_price - last_price) / last_price) * 100.0
        diff = abs(real_pct - pred_pct)

        # Grading
        grade = "✅ WIN" if diff < 2.0 else "❌ MISS"
        if "Stale" in freshness_status and grade == "❌ MISS": grade += " (Due to Latency)"

        print(f"   Actual Market: {real_pct:+.2f}%  (${real_price:.2f})")
        print(f"   Result: {grade} (Diff: {diff:.2f}%)")
    else:
        print("   Actual Market: Future data not available.")
    print("-" * 60)

# ============================================================
# 5. EXECUTION
# ============================================================
print("🔄 Initializing System...")

# Load Scaler
if os.path.exists(SCALER_PATH):
    with open(SCALER_PATH, 'rb') as f:
        feature_scaler = pickle.load(f)
else:
    feature_scaler = None
    print("⚠️ No Scaler Found!")

# Load Data Frames
sat_df = pd.read_csv(AREA_FEATURE_PATH)
sat_df['Date'] = pd.to_datetime(sat_df['Date'])
weather_df = get_area_weather(WEATHER_DIR, 'area_16')
fut_raw = pd.read_csv(FUTURES_PATH)
fut_df = engineer_futures(fut_raw)

# Merge
if weather_df is not None:
    merged = pd.merge(sat_df, weather_df, on='Date', how='left')
    merged[['GDD_Cum_7d', 'Precip_Sum_7d']] = merged[['GDD_Cum_7d', 'Precip_Sum_7d']].ffill().bfill().fillna(0)
else:
    print("⚠️ WARNING: Weather data not found for this area. Padding missing columns.")
    merged = sat_df

full_df = pd.merge(merged, fut_df, on='Date', how='left')
full_df = full_df.sort_values('Date').ffill().bfill()

# Prepare Model
img_cols = [c for c in full_df.columns if c.startswith('feat_')]
# fut_cols logic is dynamic inside the predict function, but we MUST init model with 15
model = CornETFModelFinal(len(img_cols), EXPECTED_FUT_DIM, NUM_AREAS).to(DEVICE)

if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval()
    print("✅ Model Loaded.")
else:
    print("❌ Model File Missing!")

# ============================================================
# 6. RUN THE TEST SCENARIOS
# ============================================================
scenarios = [
    # 2021: The Validation Year (Fold 1)
    ("2021-05-07", "The 2021 Rally Peak"),
    ("2021-07-01", "The Summer Correction"),

    # 2022: The Out-of-Sample Year (Fold 2)
    ("2022-03-01", "The Ukraine War Shock"),
    ("2022-06-15", "The 2022 Highs"),

    # 2023: The Crash (Fold 3)
    ("2023-01-15", "The 2023 Decline")
]

print("\n🚀 STARTING MULTI-DATE SIMULATION")
print("=" * 60)

for date, desc in scenarios:
    predict_scenario(date, desc, model, feature_scaler, full_df)

🔄 Initializing System...
✅ Model Loaded.

🚀 STARTING MULTI-DATE SIMULATION

📝 SCENARIO: The 2021 Rally Peak
   Target Date: 2021-05-07 | Last Image Seen: 2021-04-15 (✅ Fresh)
   AI Prediction: +0.80%  ($594.74)
   Actual Market: +30.97%  ($772.75)
   Result: ❌ MISS (Diff: 30.17%)
------------------------------------------------------------

📝 SCENARIO: The Summer Correction
   Target Date: 2021-07-01 | Last Image Seen: 2021-06-17 (✅ Fresh)
   AI Prediction: +0.76%  ($637.80)
   Actual Market: +13.70%  ($719.75)
   Result: ❌ MISS (Diff: 12.95%)
------------------------------------------------------------

📝 SCENARIO: The Ukraine War Shock
   Target Date: 2022-03-01 | Last Image Seen: 2022-02-14 (✅ Fresh)
   AI Prediction: +0.51%  ($659.13)
   Actual Market: +12.81%  ($739.75)
   Result: ❌ MISS (Diff: 12.29%)
------------------------------------------------------------

📝 SCENARIO: The 2022 Highs
   Target Date: 2022-06-15 | Last Image Seen: 2022-05-23 (✅ Fresh)
   AI Prediction: +0.42% 